In [42]:
# Import required libraries
import os
import json
import time
import datetime as dt
import csv
import pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Setup data directories
DATA_RAW = pathlib.Path("../data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {DATA_RAW.absolute()}")

# Load environment variables
load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded ALPHAVANTAGE_API_KEY?", bool(ALPHA_KEY))



Data directory: /Users/aman/Desktop/Bootcamp/bootcamp_aman_dhillon/project/notebooks/../data/raw
Loaded ALPHAVANTAGE_API_KEY? True


In [43]:
# Simple SPY stock data extraction with adjusted close
import requests
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
from datetime import datetime

# Load API key
load_dotenv()
POLYGON_KEY = os.getenv("POLYGON_API_KEY")

# Setup
SYMBOL = "SPY"
DATA_RAW = Path("../data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

# Get 1 year of stock data
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - pd.DateOffset(years=1)).strftime("%Y-%m-%d")

url = f"https://api.polygon.io/v2/aggs/ticker/{SYMBOL}/range/1/day/{start_date}/{end_date}"
params = {"apiKey": POLYGON_KEY}

try:
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    data = response.json()
    
    if data.get("status") in ["OK", "DELAYED"] and data.get("results"):
        df = pd.DataFrame(data["results"])
        df['date'] = pd.to_datetime(df['t'], unit='ms')
        
        # Include adjusted close (vw = volume weighted average price)
        df = df[['date', 'o', 'h', 'l', 'c', 'v', 'vw']].rename(columns={
            'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume', 'vw': 'adj_close'
        })
        
        # Save data
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"spy_stock_1year_{timestamp}.csv"
        filepath = DATA_RAW / filename
        df.to_csv(filepath, index=False)
        
        print(f"✅ Saved {len(df)} days of SPY data to: {filename}")
        print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
        print(f"Latest close: ${df['close'].iloc[-1]:.2f}")
        print(f"Latest adj_close: ${df['adj_close'].iloc[-1]:.2f}")
        
    else:
        print("❌ No data received")
        
except Exception as e:
    print(f"❌ Error: {e}")

✅ Saved 250 days of SPY data to: spy_stock_1year_20250821_165509.csv
Date range: 2024-08-21 to 2025-08-20
Latest close: $638.11
Latest adj_close: $636.80


In [82]:
# yfinance options - shortest code
import yfinance as yf

# Get options for SPY
ticker = yf.Ticker("SPY")
options = ticker.options

if options:
    # Get first expiration date
    opt = ticker.option_chain(options[0])
    
    print(f"SPY options for {options[0]}:")
    print(f"Calls: {len(opt.calls)} | Puts: {len(opt.puts)}")
    print(f"Call columns: {list(opt.calls.columns)}")
    print(f"Put columns: {list(opt.puts.columns)}")
    
    # Show sample data
    print(f"\nSample call option:")
    print(opt.calls.head(1))
else:
    print("No options found")

SPY options for 2025-08-25:
Calls: 73 | Puts: 83
Call columns: ['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'change', 'percentChange', 'volume', 'openInterest', 'impliedVolatility', 'inTheMoney', 'contractSize', 'currency']
Put columns: ['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'change', 'percentChange', 'volume', 'openInterest', 'impliedVolatility', 'inTheMoney', 'contractSize', 'currency']

Sample call option:
       contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0  SPY250825C00530000 2025-08-22 13:30:03+00:00   530.0     106.25  113.98   

      ask    change  percentChange  volume  openInterest  impliedVolatility  \
0  116.78 -9.190002      -7.960848    11.0            11           0.841798   

   inTheMoney contractSize currency  
0        True      REGULAR      USD  
